In [1]:
import sys
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')
#!pip install tqdm #--user
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
#!pip install gensim #--user
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

## Proper Doc2vec model

In [2]:
supp_df=pd.read_csv("meetup.csv")

In [3]:
supp_df=supp_df[supp_df["category"].isna()!=True]

In [4]:
cat_list=list(supp_df.category.unique())
cat_list=[str(elem) for elem in cat_list]
cat_list.sort()
cat_list=[elem.replace("/", " ") for elem in cat_list]
"""
cat_list_old= cat_list[:]
cat_list=[]
for elem in cat_list_old:
    if " " in elem:
        els=elem.split(" ")
        cat_list.append(els)
    else:
        cat_list.append(elem)
"""
cat_list

['LGBT',
 'alternative lifestyle',
 'book clubs',
 'career business',
 'cars motorcycles',
 'community environment',
 'dancing',
 'education learning',
 'fashion beauty',
 'fine arts culture',
 'fitness',
 'food drink',
 'games',
 'health wellbeing',
 'hobbies crafts',
 'language ethnic identity',
 'movements politics',
 'movies film',
 'music',
 'new age spirituality',
 'outdoors adventure',
 'paranormal',
 'parents family',
 'pets animals',
 'photography',
 'religion beliefs',
 'sci-fi fantasy',
 'singles',
 'socializing',
 'sports recreation',
 'support',
 'tech',
 'writing']

In [2]:
df=pd.read_csv("processed_description_sine_error_Porter_no_bad_words.csv")

df.head(50)

,Unnamed: 0,description,category_bin,lang_ok,processed_description
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...
2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tent introduct orient session break network au...
3,3,"<p><span>Hey explorersssss, what's up?</span><...",15,english,hey explorersssss say night view hong kong vic...
4,4,"<p>Free, unauditioned, collaborative pop choir...",18,english,unaudit collabor pop choir identifi women see ...
5,6,"<p>We provide a forum to learn about, promote ...",3,english,provid forum learn promot grow busi
6,7,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...
7,8,<p>This is an introductory meeting to get to k...,31,english,introductori meet get know fellow coder share ...
8,9,"<p>The Future of the IoT with Rajeev Suri, CEO...",31,english,futur iot rajeev suri ceo kathrin buvac chief ...
9,10,"<p><img src=""https://secure.meetupstatic.com/p...",13,english,breakthrough weight loss wk workshop lose weig...


In [8]:
df["category"]="NaN"

df["category"]= df.apply(lambda r: cat_list[r.category_bin], axis=1)
del supp_df
df.head(2)

,Unnamed: 0,description,category_bin,lang_ok,processed_description,category
0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...,singles
1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...,socializing


## Formatting as tagged doc


In [9]:
process_all=True
#process_all=True

if process_all:
    train, test = train_test_split(df, test_size=0.3, random_state=42)
else:    
    train, test = train_test_split(df[:50000], test_size=0.3, random_state=42)

train_tagged = train.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=str(r.processed_description).split(" "), tags=[r.category]), axis=1)

In [10]:
#train_tagged.values[30]

## Training dbow model

In [11]:
import multiprocessing

#cores = multiprocessing.cpu_count()
#cores
cores=4

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)

In [12]:
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 122106/122106 [00:00<00:00, 1985022.44it/s]


In [13]:
%%time
for epoch in range(10):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 122106/122106 [00:00<00:00, 4307217.39it/s]


CPU times: user 5min 15s, sys: 4.2 s, total: 5min 19s
Wall time: 1min 30s


In [14]:
def vec_for_learning(model, tagged_docs):
    #sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in tqdm(tagged_docs.values)])
    return targets, regressors

## Inferring doc vectors

In [15]:
%%time
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

100%|██████████| 52332/52332 [02:55<00:00, 297.56it/s]

CPU times: user 10min, sys: 743 ms, total: 10min
Wall time: 9min 59s


In [16]:
go=True
#go=True
if go:
    import joblib
    if process_all:
        joblib.dump(model_dbow, "model_dbow_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(X_train, "X_train_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(y_train, "y_train_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(X_test, "X_test_all_10_ep_Porter_no_bad_words.joblib")
        joblib.dump(y_test, "y_test_all_10_ep_Porter_no_bad_words.joblib")

In [19]:
go=False
#go=True

if go:
    logreg = LogisticRegression(n_jobs=5, C=1e5, solver="sag")

    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    from sklearn.metrics import accuracy_score, f1_score

    print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
    print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6514178541577478
Testing F1 score: 0.6497767054978865


## Training dmm model

In [25]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 122106/122106 [00:00<00:00, 3268678.90it/s]


In [26]:
%%time
for epoch in range(10):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|██████████| 122106/122106 [00:00<00:00, 4137546.83it/s]


CPU times: user 9min 4s, sys: 11.8 s, total: 9min 16s
Wall time: 2min 20s


In [27]:
y_train_dmm, X_train_dmm = vec_for_learning(model_dmm, train_tagged)
y_test_dmm, X_test_dmm = vec_for_learning(model_dmm, test_tagged)


100%|██████████| 52332/52332 [03:57<00:00, 220.01it/s]


In [28]:
#go=False
go=True
if go:
    import joblib
    if process_all:
        joblib.dump(model_dbow, "model_ddm_all_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_train_dmm, "X_train_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_train_dmm, "y_train_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_test_dmm, "X_test_all_ddm_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_test_dmm, "y_test_all_ddm_10_ep_Porter_no_badwords.joblib")

## Concat models

In [48]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [49]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [50]:
y_train_pair, X_train_pair = vec_for_learning(new_model, train_tagged)
y_test_pair, X_test_pair = vec_for_learning(new_model, test_tagged)

100%|██████████| 52332/52332 [07:39<00:00, 113.77it/s]


In [54]:
go=True
if go:
    import joblib
    if process_all:
        joblib.dump(new_model, "model_pair_all_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_train_pair, "X_train_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_train_pair, "y_train_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(X_test_pair, "X_test_all_pair_10_ep_Porter_no_badwords.joblib")
        joblib.dump(y_test_pair, "y_test_all_pair_10_ep_Porter_no_badwords.joblib")

## Classification

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_curve, auc

In [23]:
forest = RandomForestClassifier(n_jobs=4, n_estimators=100) 
forest = forest.fit(X_train, y_train)

In [35]:
save=True
if save:
    joblib.dump(forest, "model_rf_dbow_68acc.joblib")

In [24]:
print(forest.score(X_test, y_test))

predictions = forest.predict(X_test)

print(metrics.classification_report(y_test, predictions)) #100 features

0.6882404647252159
                          precision    recall  f1-score   support

                    LGBT       0.96      0.39      0.55       474
   alternative lifestyle       0.97      0.77      0.86       227
              book clubs       0.74      0.45      0.56       381
         career business       0.69      0.77      0.73      5124
        cars motorcycles       0.95      0.47      0.63       388
   community environment       0.76      0.44      0.56       931
                 dancing       0.83      0.76      0.80      1223
      education learning       0.77      0.48      0.59      1023
          fashion beauty       0.83      0.23      0.36       109
       fine arts culture       0.81      0.53      0.64      1188
                 fitness       0.78      0.69      0.73      2329
              food drink       0.73      0.53      0.61      1458
                   games       0.82      0.80      0.81      1472
        health wellbeing       0.67      0.73      0.70 

In [44]:
forest1 = RandomForestClassifier(n_jobs=6, n_estimators=200) 
forest1 = forest.fit(X_train_dmm, y_train_dmm)

In [45]:
forest1.score(X_test_dmm, y_test_dmm)

0.511083084919361

In [52]:
save=True
if save:
    joblib.dump(forest1, "model_rf_dmm_52acc.joblib")

In [47]:
predictions = forest1.predict(X_test_dmm)
print(metrics.classification_report(y_test_dmm, predictions)) #100 features

                          precision    recall  f1-score   support

                    LGBT       1.00      0.20      0.33       474
   alternative lifestyle       0.90      0.69      0.78       227
              book clubs       0.95      0.11      0.19       381
         career business       0.45      0.65      0.54      5124
        cars motorcycles       0.99      0.17      0.29       388
   community environment       0.96      0.18      0.30       931
                 dancing       0.89      0.39      0.55      1223
      education learning       0.87      0.26      0.39      1023
          fashion beauty       0.92      0.10      0.18       109
       fine arts culture       0.92      0.22      0.35      1188
                 fitness       0.72      0.40      0.51      2329
              food drink       0.83      0.24      0.37      1458
                   games       0.85      0.46      0.60      1472
        health wellbeing       0.43      0.64      0.51      4998
         

### Final model

In [55]:
forest_pair = RandomForestClassifier(n_jobs=6) 
forest_pair = forest.fit(X_train_pair, y_train_pair)

In [56]:
forest_pair.score(X_test_pair, y_test_pair)#100feat

0.6799281510356951

In [57]:
save=True
if save:
    joblib.dump(forest_pair, "model_rf_pair.joblib")

In [58]:
predictions = forest_pair.predict(X_test_pair)
print(metrics.classification_report(y_test_pair, predictions)) #100 features

                          precision    recall  f1-score   support

                    LGBT       0.96      0.38      0.54       474
   alternative lifestyle       0.97      0.75      0.85       227
              book clubs       0.78      0.45      0.57       381
         career business       0.68      0.77      0.72      5124
        cars motorcycles       0.98      0.44      0.60       388
   community environment       0.79      0.43      0.56       931
                 dancing       0.83      0.75      0.79      1223
      education learning       0.78      0.45      0.57      1023
          fashion beauty       0.92      0.22      0.36       109
       fine arts culture       0.81      0.52      0.64      1188
                 fitness       0.78      0.68      0.73      2329
              food drink       0.74      0.52      0.61      1458
                   games       0.82      0.80      0.81      1472
        health wellbeing       0.66      0.73      0.69      4998
         

In [20]:
forest.score(X_test, y_test)

predictions = forest.predict(X_test)


print(metrics.classification_report(y_test, predictions)) #100 features

0.6302453565695941